<a href="https://colab.research.google.com/github/QiaoXingLi/-1/blob/master/Concil%20Web3%20UseCase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Example

Install the council-ai library

In [ ]:
!pip install council-ai

Import Council.

In [ ]:
from council.chains import Chain
from council.skills import LLMSkill
from council.llm import OpenAILLM, OpenAILLMConfiguration


Setup API keys and use it to setup the LLM (here: OpenAILLM).

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = 'your api key here'

openai_llm = OpenAILLM(config=OpenAILLMConfiguration.from_env())


Create your first Hello World Skill and Wrap it in a Chain.


In [ ]:
prompt = "You are responding to every prompt with a short poem titled hello world"
hw_skill = LLMSkill(llm=openai_llm, system_prompt=prompt)
hw_chain = Chain(name="Hello World", description="Answers with a poem about titled Hello World", runners=[hw_skill])


Create a second Skill (that responds only with Emojis).


In [ ]:
prompt = "You are responding to every prompt with an emoji that best addresses the question asked or statement made"
em_skill = LLMSkill(llm=openai_llm, system_prompt=prompt)
em_chain = Chain(name="Emoji Agent", description="Responds to every prompt with an emoji that best fits the prompt",
                 runners=[em_skill])

Create a Controller to route prompts to chains. Here we use the straight-forward LLMController in which an LLM instance is tasked to make a routing decision.


In [ ]:
from council.controllers import LLMController

controller = LLMController(llm=openai_llm, response_threshold=5)


Create an Evaluator. Here, we use an LLMEvaluator in which an LLM is tasked to evaluate each response received.


In [ ]:
from council.evaluators import LLMEvaluator

evaluator = LLMEvaluator(llm=openai_llm)


Finalize setup of the Hello World first Agent by combining all components created.


In [ ]:
from council.agents import Agent

agent = Agent(controller=controller, chains=[hw_chain, em_chain], evaluator=evaluator)


Now, we are ready to invoke the agent.


In [ ]:
result = agent.execute_from_user_message("hello world?!")
print(result.best_message.message)
